In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import tensorflow as tf

# CNN metadata:

In [3]:
cle = pd.read_csv('cle_metadata_cnn.csv')
vir = pd.read_csv('vir_metadata_cnn.csv')
hun = pd.read_csv('hun_metadata_cnn.csv')
swi = pd.read_csv('swi_metadata_cnn.csv')

In [4]:
data = pd.concat([cle,vir,hun,swi])
data

,0,1,2,3,4,5,6,7,8,9,...,7.3,8.3,9.3,10.3,11.3,12.3,13.3,14.3,15.3,num
0,0.0,0.850346,0.700405,0.427932,0.631793,0.720650,0.805797,0.555187,0.908639,0.0,...,1.858736,0.0,0.0,1.280513,0.0,0.0,0.0,0.0,2.336825,0.00
1,0.0,0.900102,1.238794,0.455062,1.067564,1.226179,0.716775,0.488568,0.723180,0.0,...,1.948065,0.0,0.0,1.372884,0.0,0.0,0.0,0.0,2.425866,0.00
2,0.0,0.202576,1.539479,0.914987,1.211709,1.484435,0.342889,1.055182,1.424387,0.0,...,1.634343,0.0,0.0,1.112418,0.0,0.0,0.0,0.0,2.043138,0.00
3,0.0,0.664326,1.246355,0.639915,0.900147,1.269826,0.557118,0.816253,1.045733,0.0,...,1.868872,0.0,0.0,1.317322,0.0,0.0,0.0,0.0,2.268249,0.00
4,0.0,0.961716,0.693969,0.462445,0.580937,0.851637,0.836276,0.611334,0.868884,0.0,...,1.783576,0.0,0.0,1.222539,0.0,0.0,0.0,0.0,2.169000,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,0.0,2.046181,0.079782,0.000000,0.078642,0.192834,1.641056,0.000000,0.000000,0.0,...,2.278238,0.0,0.0,1.561664,0.0,0.0,0.0,0.0,2.821322,0.75
119,0.0,1.831266,0.296626,0.000000,0.375618,0.379790,1.366388,0.000000,0.000000,0.0,...,2.111198,0.0,0.0,1.465579,0.0,0.0,0.0,0.0,2.593416,0.50
120,0.0,2.047585,0.057311,0.000000,0.123829,0.080624,1.714231,0.000000,0.000000,0.0,...,2.132200,0.0,0.0,1.489336,0.0,0.0,0.0,0.0,2.642818,1.00
121,0.0,2.216803,0.000000,0.000000,0.000000,0.000000,1.804639,0.000000,0.000000,0.0,...,2.032758,0.0,0.0,1.430741,0.0,0.0,0.0,0.0,2.525313,0.25


In [5]:
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Y_train_binary = Y_train.apply(lambda x: 1 if x > 0 else 0)
Y_test_binary = Y_test.apply(lambda x: 1 if x > 0 else 0)

In [ ]:
X_train; X_test; Y_train_binary; Y_test_binary

# CNN

In [21]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

# define the model architecture
model = Sequential()
model.add(Conv1D(filters=256, kernel_size=3, activation='relu', input_shape=(64,1)))
model.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
model.fit(X_train, Y_train_binary, epochs=1000, batch_size=32,callbacks=[callback])
Y_pred = model.predict(X_test).argmax(axis=1)
    
cm = confusion_matrix(Y_pred, Y_test_binary)
print(cm)
print(classification_report(Y_test_binary, Y_pred, digits=4))

Epoch 1/1000
20/20 [==============================] - 1s 38ms/step - loss: 0.4556 - accuracy: 0.8409
Epoch 2/1000
20/20 [==============================] - 1s 37ms/step - loss: 0.3599 - accuracy: 0.8539
Epoch 3/1000
20/20 [==============================] - 1s 38ms/step - loss: 0.3470 - accuracy: 0.8604
Epoch 4/1000
20/20 [==============================] - 1s 40ms/step - loss: 0.3586 - accuracy: 0.8539
Epoch 5/1000
20/20 [==============================] - 1s 39ms/step - loss: 0.3384 - accuracy: 0.8669
Epoch 6/1000
20/20 [==============================] - 1s 39ms/step - loss: 0.3403 - accuracy: 0.8766
Epoch 7/1000
20/20 [==============================] - 1s 36ms/step - loss: 0.3358 - accuracy: 0.8766
Epoch 8/1000
20/20 [==============================] - 1s 39ms/step - loss: 0.3347 - accuracy: 0.8718
Epoch 9/1000
20/20 [==============================] - 1s 39ms/step - loss: 0.3380 - accuracy: 0.8685
Epoch 10/1000
20/20 [==============================] - 1s 38ms/step - loss: 0.3351 - accura

In [22]:
model.save('../Models/Meta_Only/CNNMeta_CNN.h5')

# Test on Each Data Set

In [23]:
def Test(path,model_name):
    df = pd.read_csv(path)
    X = df.iloc[:,:-1]
    y= df.iloc[:,-1]
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    #binarize the target
    Y_train_binary = Y_train.apply(lambda x: 1 if x > 0 else 0)
    Y_test_binary = Y_test.apply(lambda x: 1 if x > 0 else 0)

    model = tf.keras.models.load_model(model_name)

    Y_pred = model.predict(X_test).argmax(axis=1)
    
    cm = confusion_matrix(Y_pred, Y_test_binary)
    print(cm)
    print(classification_report(Y_test_binary, Y_pred, digits=4))

# Cleveland

In [24]:
path = 'cle_metadata_cnn.csv'
model = '../Models/Meta_Only/CNNMeta_CNN.h5'
Test(path,model)

4/4 [==============================] - 0s 7ms/step
[[30  4]
 [22 44]]
              precision    recall  f1-score   support

           0     0.8824    0.5769    0.6977        52
           1     0.6667    0.9167    0.7719        48

    accuracy                         0.7400       100
   macro avg     0.7745    0.7468    0.7348       100
weighted avg     0.7788    0.7400    0.7333       100



# Virginia

In [25]:
path = 'vir_metadata_cnn.csv'
model = '../Models/Meta_Only/CNNMeta_CNN.h5'
Test(path,model)

3/3 [==============================] - 0s 6ms/step
[[ 6  1]
 [ 9 50]]
              precision    recall  f1-score   support

           0     0.8571    0.4000    0.5455        15
           1     0.8475    0.9804    0.9091        51

    accuracy                         0.8485        66
   macro avg     0.8523    0.6902    0.7273        66
weighted avg     0.8497    0.8485    0.8264        66



# Hungarian

In [26]:
path = 'hun_metadata_cnn.csv'
model = '../Models/Meta_Only/CNNMeta_CNN.h5'
Test(path,model)

4/4 [==============================] - 0s 7ms/step
[[50  4]
 [10 34]]
              precision    recall  f1-score   support

           0     0.9259    0.8333    0.8772        60
           1     0.7727    0.8947    0.8293        38

    accuracy                         0.8571        98
   macro avg     0.8493    0.8640    0.8532        98
weighted avg     0.8665    0.8571    0.8586        98



# Switzerland

In [27]:
path = 'swi_metadata_cnn.csv'
model = '../Models/Meta_Only/CNNMeta_CNN.h5'
Test(path,model)

2/2 [==============================] - 0s 5ms/step
[[ 1  2]
 [ 2 36]]
              precision    recall  f1-score   support

           0     0.3333    0.3333    0.3333         3
           1     0.9474    0.9474    0.9474        38

    accuracy                         0.9024        41
   macro avg     0.6404    0.6404    0.6404        41
weighted avg     0.9024    0.9024    0.9024        41



# DNN

In [6]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

# define the model architecture
model = Sequential()
model.add(Dense(64, input_shape=(64,), activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
model.fit(X_train, Y_train_binary, epochs=1000, batch_size=32,callbacks=[callback])
Y_pred = model.predict(X_test).argmax(axis=1)
    
cm = confusion_matrix(Y_pred, Y_test_binary)
print(cm)
print(classification_report(Y_test_binary, Y_pred, digits=4))

Epoch 1/1000
20/20 [==============================] - 0s 1000us/step - loss: 0.4508 - accuracy: 0.7922
Epoch 2/1000
20/20 [==============================] - 0s 789us/step - loss: 0.3743 - accuracy: 0.8571
Epoch 3/1000
20/20 [==============================] - 0s 789us/step - loss: 0.3509 - accuracy: 0.8571
Epoch 4/1000
20/20 [==============================] - 0s 842us/step - loss: 0.3372 - accuracy: 0.8571
Epoch 5/1000
20/20 [==============================] - 0s 842us/step - loss: 0.3439 - accuracy: 0.8571
Epoch 6/1000
20/20 [==============================] - 0s 790us/step - loss: 0.3413 - accuracy: 0.8653
Epoch 7/1000
20/20 [==============================] - 0s 790us/step - loss: 0.3418 - accuracy: 0.8669
Epoch 8/1000
20/20 [==============================] - 0s 789us/step - loss: 0.3386 - accuracy: 0.8604
Epoch 9/1000
10/10 [==============================] - 0s 667us/step
[[ 97  15]
 [ 30 162]]
              precision    recall  f1-score   support

           0     0.8661    0.7638    

In [7]:
model.save('../Models/Meta_Only/CNNMeta_DNN.h5')

# RNN

In [14]:
from keras.layers import Dense, SimpleRNN

In [15]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

# define the model architecture
model = Sequential()
model.add(SimpleRNN(units=64, return_sequences=True, input_shape=(64, 1)))
model.add(SimpleRNN(units=32, return_sequences=True))
model.add(SimpleRNN(units=16))
model.add(Dense(units=2, activation='sigmoid'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
model.fit(X_train, Y_train_binary, epochs=1000, batch_size=32,callbacks=[callback])
Y_pred = model.predict(X_test).argmax(axis=1)
    
cm = confusion_matrix(Y_pred, Y_test_binary)
print(cm)
print(classification_report(Y_test_binary, Y_pred, digits=4))

Epoch 1/1000
7/7 [==============================] - 1s 11ms/step - loss: 0.7239 - accuracy: 0.5419
Epoch 2/1000
7/7 [==============================] - 0s 11ms/step - loss: 0.5772 - accuracy: 0.7291
Epoch 3/1000
7/7 [==============================] - 0s 11ms/step - loss: 0.4849 - accuracy: 0.7783
Epoch 4/1000
7/7 [==============================] - 0s 10ms/step - loss: 0.5141 - accuracy: 0.7685
Epoch 5/1000
7/7 [==============================] - 0s 12ms/step - loss: 0.3981 - accuracy: 0.8522
Epoch 6/1000
7/7 [==============================] - 0s 10ms/step - loss: 0.3741 - accuracy: 0.8719
Epoch 7/1000
7/7 [==============================] - 0s 10ms/step - loss: 0.3338 - accuracy: 0.8867
Epoch 8/1000
7/7 [==============================] - 0s 11ms/step - loss: 0.3448 - accuracy: 0.8818
Epoch 9/1000
7/7 [==============================] - 0s 11ms/step - loss: 0.3360 - accuracy: 0.8768
Epoch 10/1000
7/7 [==============================] - 0s 11ms/step - loss: 0.3401 - accuracy: 0.8768
Epoch 11/

In [16]:
model.save('../Models/Meta_Only/CNNMeta_RNN.h5')

# Test on Each Data Set

In [17]:
def Test(path,model_name):
    df = pd.read_csv(path)
    X = df.iloc[:,:-1]
    y= df.iloc[:,-1]
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    #binarize the target
    Y_train_binary = Y_train.apply(lambda x: 1 if x > 0 else 0)
    Y_test_binary = Y_test.apply(lambda x: 1 if x > 0 else 0)

    model = tf.keras.models.load_model(model_name)

    Y_pred = model.predict(X_test).argmax(axis=1)
    
    cm = confusion_matrix(Y_pred, Y_test_binary)
    print(cm)
    print(classification_report(Y_test_binary, Y_pred, digits=4))

# Cleveland

In [18]:
path = 'cle_metadata_cnn.csv'
model = '../Models/Meta_Only/CNNMeta_RNN.h5'
Test(path,model)

4/4 [==============================] - 0s 4ms/step
[[39  4]
 [13 44]]
              precision    recall  f1-score   support

           0     0.9070    0.7500    0.8211        52
           1     0.7719    0.9167    0.8381        48

    accuracy                         0.8300       100
   macro avg     0.8395    0.8333    0.8296       100
weighted avg     0.8422    0.8300    0.8292       100



# Virginia

In [19]:
path = 'vir_metadata_cnn.csv'
model = '../Models/Meta_Only/CNNMeta_RNN.h5'
Test(path,model)

3/3 [==============================] - 0s 4ms/step
[[ 2  1]
 [13 50]]
              precision    recall  f1-score   support

           0     0.6667    0.1333    0.2222        15
           1     0.7937    0.9804    0.8772        51

    accuracy                         0.7879        66
   macro avg     0.7302    0.5569    0.5497        66
weighted avg     0.7648    0.7879    0.7283        66



# Hungarian

In [20]:
path = 'hun_metadata_cnn.csv'
model = '../Models/Meta_Only/CNNMeta_RNN.h5'
Test(path,model)

4/4 [==============================] - 0s 4ms/step
[[34  3]
 [26 35]]
              precision    recall  f1-score   support

           0     0.9189    0.5667    0.7010        60
           1     0.5738    0.9211    0.7071        38

    accuracy                         0.7041        98
   macro avg     0.7463    0.7439    0.7041        98
weighted avg     0.7851    0.7041    0.7034        98



# Switzerland

In [21]:
path = 'swi_metadata_cnn.csv'
model = '../Models/Meta_Only/CNNMeta_RNN.h5'
Test(path,model)

2/2 [==============================] - 0s 5ms/step
[[ 1  1]
 [ 2 37]]
              precision    recall  f1-score   support

           0     0.5000    0.3333    0.4000         3
           1     0.9487    0.9737    0.9610        38

    accuracy                         0.9268        41
   macro avg     0.7244    0.6535    0.6805        41
weighted avg     0.9159    0.9268    0.9200        41

